# Panorama Stitching

In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
img1 = cv2.imread('./images/img1.jpeg')
img2 = cv2.imread('./images/img2.jpeg')

img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(img1[:,:,::-1])
ax[1].imshow(img2[:,:,::-1])

## Находим ключевые точки и дескрипторы

In [ ]:
# https://en.wikipedia.org/wiki/Oriented_FAST_and_rotated_BRIEF

# инициализируем детектор точек
orb = cv2.ORB_create()

# запускаем поиск точек и вычисление дескрипторов
kp1, dsc1 = orb.detectAndCompute(img1_gray, None)
kp2, dsc2 = orb.detectAndCompute(img2_gray, None)

In [ ]:
img1_with_kps = cv2.drawKeypoints(img1, kp1, None)

plt.figure(figsize=(20,10))
plt.imshow(img1_with_kps[:,:,::-1])

## Матчим ключевые точки. Убираем плохие матчи.

In [ ]:
matcher = cv2.BFMatcher()
matches = matcher.match(dsc1, dsc2)

# сортируем результат по убыванию расстояния
matches = sorted(matches, key = lambda x: x.distance)

In [ ]:
imgs_with_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None, flags=2)

plt.figure(figsize=(20,10))
plt.imshow(imgs_with_matches[:,:,::-1])

## Находим projective transform

In [ ]:
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches[:40]])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches[:40]])

In [ ]:
M, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

## Преобразовываем изображения

In [ ]:
h, w = img1.shape[0], img1.shape[1]+img2.shape[1]

In [ ]:
panorama = cv2.warpPerspective(img2, M, (w,h))

In [ ]:
panorama[0:img1.shape[0], 0:img1.shape[1]] = img1

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(panorama[:,:,::-1])